In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model = "meta-llama/Llama-2-7b-hf"
peft_model = "FinGPT/fingpt-mt_llama2-7b_lora"

tokenizer = AutoTokenizer.from_pretrained(base_model, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", load_in_4bit=True, torch_dtype=torch.float16)

model = PeftModel.from_pretrained(model, peft_model, device_map="auto", torch_dtype=torch.float16)

model = model.eval()

/home/carlos/Documents/git/llms/.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.10s/it]


In [11]:
import sys
sys.path.append('../../benchmarks/')

from fiqa import test_fiqa

dataset = test_fiqa(model, tokenizer)



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: @gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?
Answer: 


Total len: 150. Batchsize: 8. Total steps: 19


  0%|          | 0/19 [00:00<?, ?it/s]/home/carlos/Documents/git/llms/.env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2624: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
100%|██████████| 19/19 [00:09<00:00,  1.93it/s]

Acc: 0.8733333333333333. F1 macro: 0.8728695792562884. F1 micro: 0.8733333333333333. F1 weighted (BloombergGPT): 0.8728695792562883. 


In [13]:
dataset.to_csv('../../results/financial/llama2-7b-fingpt.csv')

In [12]:
import pandas as pd
dataset#.query('new_out == 1')
pd.set_option('display.max_colwidth', None)
dataset["out_text"]
dataset

,input,output,instruction,context,target,out_text,new_out
0,$HCP Come to the party and buy this -gonna give solid gains and a dividend $$$$$$,0,What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.,Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: $HCP Come to the party and buy this -gonna give solid gains and a dividend $$$$$$\nAnswer:,0,positive</s>,0
1,@gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?,2,What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.,Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: @gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?\nAnswer:,2,negative</s>,2
2,Japan's Asahi to submit bid next week for SABMiller's Grolsch and Peroni - Yomiuri,0,What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.,Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: Japan's Asahi to submit bid next week for SABMiller's Grolsch and Peroni - Yomiuri\nAnswer:,0,positive</s>,0
3,"Tesla Motors recalls 2,700 Model X SUVs $TSLA https://t.co/F55dx4aegI",2,What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.,"Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: Tesla Motors recalls 2,700 Model X SUVs $TSLA https://t.co/F55dx4aegI\nAnswer:",2,negative</s>,2
4,CRH's concrete bid for Holcim Lafarge assets,0,What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.,Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: CRH's concrete bid for Holcim Lafarge assets\nAnswer:,0,positive</s>,0
...,...,...,...,...,...,...,...
145,Intertek swings to ÃÂ£347 mln loss on oil's slump,2,What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.,Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: Intertek swings to ÃÂ£347 mln loss on oil's slump\nAnswer:,2,negative</s>,2
146,RT @jan $ARNA Don't think buyout rumor strong enough too be driving this up just yet...when it is this may fly like aapl...// fly how high?,1,What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.,Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: RT @jan $ARNA Don't think buyout rumor strong enough too be driving this up just yet...when it is this may fly like aapl...// fly how high?\nAnswer:,1,neutral</s>,1
147,Barclays appoints JPMorgan's Paul Compton as new COO,1,What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.,Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: Barclays appoints JPMorgan's Paul Compton as new COO\nAnswer:,1,neutral</s>,1
148,$AAPL Now I'm glad I got stopped out of my $113 weeklies when I did. Will re-enter next Monday for the #earnings run-up.,1,What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.,Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: $AAPL Now I'm glad I got stopped out of my $113 weeklies when I did. Will re-enter next Monday for the #earnings run-up.\nAnswer:,1,neutral</s>,1


In [1]:
import pandas as pd

df = pd.read_csv('../../results/financial/llama2-7b-fingpt.csv', index_col=0)

true, pred = df["target"], df["new_out"]


import sys
sys.path.append('../../')
from metrics import metrics

metrics(true, pred)

Precision: 0.873, Recall: 0.873, F1: 0.873, Accuracy: 0.873
